# 6.2 特徴選択および特徴量の重要度

モデルの予測に寄与しない特徴を判定することで、それらを取り除き精度向上や計算時間の短縮が可能となる。

ここでは、以下の 3 つの方法を用いてそのような特徴量の判定方法を説明する。

1. 単変量統計を用いる方法
2. 特徴量の重要度を用いる方法
3. 反復して探索する方法

In [ ]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

# train_x は学習データ、train_y は目的変数、test_x はテストデータ
train = pd.read_csv('../input/sample-data/train_preprocessed_onehot.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('../input/sample-data/test_preprocessed_onehot.csv')

## 6.2.1 単変量統計を用いる方法

各特徴量と目的変数から何らかの統計量を計算し、その統計量の順序で特徴量を選択することを考える。

その中で、特徴量と目的変数の 1:1 の関係に着目した単変量統計について考える。

### 相関係数

各特徴量 $x_i$ と目的変数 $y_i$ の相関係数 (ピアソンの積率相関係数)

$$
\rho = \frac{\sum_i(x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_i(x_i - {x})^2\sum_i(y_i - \bar{y})^2}}
$$

を計算してその絶対値の大きい順に選択する。  
線形以外の関係性を捉えることはできないので注意。

In [ ]:
# ---------------------------------
# 相関係数
# ---------------------------------
import scipy.stats as st

# 相関係数
corrs = []
for c in train_x.columns:
    corr = np.corrcoef(train_x[c], train_y)[0, 1]
    corrs.append(corr)
corrs = np.array(corrs)

# 重要度の上位を出力する（上位5個まで）
# np.argsortを使うことで、値の順序のとおりに並べたインデックスを取得できる
idx = np.argsort(np.abs(corrs))[::-1]
top_cols, top_importances = train_x.columns.values[idx][:5], corrs[idx][:5]
print(top_cols, top_importances)

もう一つ相関を計る統計量として、スピアマンの順位相関係数がある。  
ピアソンの相関係数では特徴量と考えていた $x_i, y_i$ を何らかの指標に基づいた単なる順位と置き換えてあげて

$$
\sum_i x_i = \sum_i y_i = \frac{n(n+1)}{2}, \quad 
\sum_i x_i^2 = \sum_i y_i^2 = \frac{n(n+1)(2n+1)}{6},
$$

などとしてあげると、ピアソンの相関係数はそのまま以下のように変形できる。

$$
\rho = 1 - \frac{6}{n(n^2-1)} \sum_{i=1}^n (x_i - y_i)^2
$$

これをスピアマンの順位相関係数と呼び、 例えば生徒の英語の成績の順位と数学の成績の順位に相関があるかなどを計ったりする。  
計算してみると、順位が全て同じときは $1$ に、逆に順位が真逆のときは $-1$ になることがわかる。

同率の順位のものが多くある場合はもう少し複雑な式が用いられる。


In [ ]:
# スピアマンの順位相関係数
corrs_sp = []
for c in train_x.columns:
    corr_sp = st.spearmanr(train_x[c], train_y).correlation
    corrs_sp.append(corr_sp)
corrs_sp = np.array(corrs_sp)

idx2 = np.argsort(np.abs(corrs_sp))[::-1]
top_cols2, top_importances2 = train_x.columns.values[idx][:5], corrs_sp[idx][:5]
print(top_cols2, top_importances2)

### カイ二乗統計量

各特徴量と目的変数について独立性の検定を行い、カイ二乗検定の統計量を計算する。  

$i$ 番目の特徴量について、target が $j$ である $k (k=1, 2, ..., n)$ 番目のデータを $f_{ijk}$ として、観測度数 $O$ と期待度数 $E$ を定義する。

$$
O_{ij} = \sum_{k}^n f_{ijk}, \quad E_{ij} = \frac{({\rm number\ of\ data\ with\ target}\ j)}{n}.
$$

もし $i$ 番目の特徴量と target が独立であれば、以下の統計量は自由度 $n-1$ の $\chi^2$ 分布に従うはずである。

$$
\chi^2_i \equiv \sum_j \frac{(O_{ij} - E_{ij})^2}{E_{ij}} \sim \chi^2(n-1).
$$

従って、以上の統計量を求めることで、独立かどうかの判定を行うことができる。

- 特徴量の値でスケールされるので、min-max scaling などを事前に行う
- 独立性の検定は一般に頻度を表す特徴量について用いられるが、機械学習の文脈だと連続値に対しても応用されている  
  従って、分類タスクでの非負の特徴量についてのみ使える

python では `sklearn.feature_selection` を使えば簡単に実装できる。

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

# カイ二乗統計量
x = MinMaxScaler().fit_transform(train_x)
c2, _ = chi2(x, train_y)

# 重要度の上位を出力する（上位5個まで）
idx = np.argsort(c2)[::-1]
top_cols, top_importances = train_x.columns.values[idx][:5], corrs[idx][:5]
print(top_cols, top_importances)

### 相互情報量

各特徴量と目的変数の相互情報量

$$
I(X; Y) = \int_X \int_Y p(x, y) \log \frac{p(x,y)}{p(x)p(y)} dxdy
$$

を計算し、大きいものから特徴量を選択する。

一般にデータから (同時) 確率分布を求めるのはそんなに簡単ではないと思うが、やり方は [sklearn の reference](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html) を参照してみてください。


In [ ]:
# ---------------------------------
# 相互情報量
# ---------------------------------
from sklearn.feature_selection import mutual_info_classif

# 相互情報量
mi = mutual_info_classif(train_x, train_y)

# 重要度の上位を出力する（上位5個まで）
idx = np.argsort(mi)[::-1]
top_cols, top_importances = train_x.columns.values[idx][:5], corrs[idx][:5]
print(top_cols, top_importances)

最後に、学習データ全体で特徴量選択をしてしまうと、本来は目的変数と関係ないにもかかわらずたまたま学習データで偏りが出ている特徴量が有効な特徴量と認識されてしまうことがある。  
そのため、特徴量選択も out-of-fold で行ったほうがよいいこともあると覚えておくとよい。

## 6.2.2 特徴量の重要度を用いる方法

モデルから出力される特徴量の重要度を用いて特徴量を選択する方法を紹介する。

### ランダムフォレストの特徴量の重要度

Random forest の重要度は、分岐を作成するときの基準となる値の減少によって計算される。  
回帰では二乗誤差、分類ではジニ不純度 
$$
\sum_i^{n} p_i (1-p_i)
$$
($p_i$ はあるノードにおけるターゲットラベル $i$ の頻度)
を元にして計算される。

In [ ]:
# ---------------------------------
# ランダムフォレストの特徴量の重要度
# ---------------------------------
from sklearn.ensemble import RandomForestClassifier

# ランダムフォレスト
clf = RandomForestClassifier(n_estimators=10, random_state=71)
clf.fit(train_x, train_y)
fi = clf.feature_importances_

# 重要度の上位を出力する
idx = np.argsort(fi)[::-1]
top_cols, top_importances = train_x.columns.values[idx][:5], fi[idx][:5]
print('random forest importance')
print(top_cols, top_importances)

### GBDT の特徴量の重要度

xgboost では以下の 3 つの種類の特徴量の重要度を出力できる。

- ゲイン: その特徴量の分岐により得た目的関数の減少
- カバー: その特徴量により分岐させられたデータの数
- 頻度: その特徴量が分岐に現れた回数

python の default では頻度が出力されるが、ゲインを出力したほうが良い。

連続変数やカテゴリの多いカテゴリ変数は分岐の候補が多いため上位になりやすかったりする。  
そのため、バラつきを考慮したりランダムな値からなる特徴量と比較することが有効。  
例えば、cross validation の fold 間での標準偏差/平均を計算し、変動係数が小さい順に特徴量を選択する手法がある。

In [ ]:
# ---------------------------------
# xgboostの特徴量の重要度
# ---------------------------------
import xgboost as xgb

# xgboost
dtrain = xgb.DMatrix(train_x, label=train_y)
params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
num_round = 50
model = xgb.train(params, dtrain, num_round)

# 重要度の上位を出力する
fscore = model.get_score(importance_type='total_gain')  # or 'total_cover'
fscore = sorted([(k, v) for k, v in fscore.items()], key=lambda tpl: tpl[1], reverse=True)
print('xgboost importance')
print(fscore[:5])



### その他の手法

#### permutation importance

モデルを学習した後、validation data のある特徴量をシャッフルした場合の予測とシャッフルしていない予測を比較し、予測精度の落ち具合から特徴量の重要度を判定する方法。  
`eli4` というライブラリが使える他、random forest の場合、学習データのサンプリングから外れた out-of-bag と呼ばれるデータを用いて `rfpimp` などのモジュールにより計算することができる。

#### null importance

目的変数をシャッフルして学習させた場合のモデルの重要度を null importance として基準とし、目的変数をシャッフルさせていない通常の重要度と比較して特徴量を選択する手法。  
null importance はシャッフルによって変わるため、数十回繰り返して統計量を用いる。

実装例は Home Credit Default Risk の kaggle kenel [Feature Selection with Null Importances](https://www.kaggle.com/ogrellier/feature-selection-with-null-importances) を参考。

#### boruta

それぞれの特徴量をシャッフルしたデータ shadow feature を元のデータに加えて random forest で学習を行い、それぞれの特徴量の重要度が全ての shadow feature より大きいものを記録する。  
これを何度か繰り返し、shadow feature より重要とは言えない特徴量を除外していく。

ライブラリ [`BorutaPy`](https://danielhomola.com/boruta_py) が公開されており、実装例は kaggle kernel [Boruta feature elimination](https://www.kaggle.com/tilii6/boruta-feature-elimination)を参考のこと。

#### 特徴量を大量生成してからの特徴量選択

機械的に特徴量を大量生成してから特徴量選択をする手法。

#### xgbfir

xgboost のモデルから決定木分岐の情報を抽出して特徴量の重要度を出力するライブラリ。  
2016 以降更新されてなさそうなので、使うことはなさそう...？

## 5.2.3 反復して探索する方法

特徴量の組み合わせを変えて学習を繰り返し、その精度などを用いて探索する手法。  
時間もかかるしあまりやることはないような気がするので省略。